In [3]:
!git clone https://github.com/07536245048/EC1B1-coursework.git
#import packages to perform graphical and other functions later and read the downloaded data in Pandas dataframes, setting the Japan data and USA data to different variables
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
df_Japan = pd.read_excel('Japan.xlsx')
df_USA = pd.read_excel('US.xlsx')




xcode-select: note: no developer tools were found at '/Applications/Xcode.app', requesting install. Choose an option in the dialog to download the command line developer tools.


FileNotFoundError: [Errno 2] No such file or directory: 'Japan.xlsx'

In [ ]:
#Change the header titles of the dataset with .rename() to variables that are more succint and comprehensive
df_Japan.rename(columns = {'Unnamed: 0': 'Date', 'Economic Activity, Industrial Production, Index': 'Japan Industrial Production', 'Prices, Consumer Price Index, All items, Index': 'Japan CPI', 'Exchange Rates, National Currency Per U.S. Dollar, Period Average, Rate': 'Exchange Rates Per U.S. Dollar', 'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar' : 'Japan International Reserves and Liquidity'}, inplace = True)
df_USA.rename(columns = {'Unnamed: 0': 'Date', 'Prices, Consumer Price Index, All items, Index': 'USA CPI', 'International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar' : 'USA International Reserves and Liquidity'}, inplace = True)



In [ ]:
#Merge the two data sets at the month level with .merge() and assign to variable df_both
df_both = pd.merge(df_Japan, df_USA, on = ['Date'])



In [ ]:
#Convert all numerical data to floats using .astype so that we can perform quantitative functions with them
df_both = df_both.astype({'Japan Industrial Production':'float','Exchange Rates Per U.S. Dollar':'float','Japan CPI':'float','USA CPI':'float'})


In [ ]:
#Use the datetime function to format the dates
df_both['Date'] = pd.to_datetime(df_both['Date']).dt.date



In [ ]:
#Create a variable for log exchange rate using np.log() of Exchange Rates variable
df_both['Japan Log Exchange Rate'] = np.log(df_both['Exchange Rates Per U.S. Dollar'])

In [ ]:
#Create a variable for the monthly change in log exchange rate using .diff(). Change in nominal ER can be calculated by doing difference in logs on a monthly basis with periods=1
df_both['Japan Monthly Nominal ER Growth'] = df_both['Japan Log Exchange Rate'].diff()



Create a variable for Japan Inflation Monthly Change. Do this by taking the CPI for time period 1 using .shift() and CPI for time period 0. Inflation is the growth in CPI between two consecutive periods so divide the difference in CPI for the two periods by the CPI in period 0. Finally, use .drop() to remove the variable created for CPI in period 1. This method will repeat the process for all 2 consecutive time periods across the 30 years.
Could also use .diff() method for log of CPI

In [ ]:
#Create a variable for Japan Inflation Monthly Change. Description in line above
df_both['Japan CPI t+1'] = df_both['Japan CPI'].shift()
CPI_t = df_both['Japan CPI']
CPI_t1 = df_both['Japan CPI t+1']
df_both['Japan Inflation Monthly Change'] = (CPI_t - CPI_t1)/CPI_t1
df_both.drop('Japan CPI t+1', inplace=True, axis=1)




In [ ]:
#Create a variable for Log Industrial Production using np.log() of Indsutrial Production variable
df_both['Japan Log Industrial Production'] = np.log(df_both['Japan Industrial Production'])


Create a variable for monthly production growth. Do this by taking the production for time period 1 using .shift() and production for time period 0. Production growth is the growth in production between two consecutive periods so divide the difference in production for the two periods by the production in period 0. Finally, use .drop() to remove the variable created for production in period 1. This method will repeat the process for all 2 consecutive time periods across the 30 years

In [ ]:
#Create a variable for monthly production growth. Description in line above
df_both['Industrial Production t+1'] = df_both['Japan Industrial Production'].shift()
Prod_t = df_both['Japan Industrial Production']
Prod_t1 = df_both['Industrial Production t+1']
df_both['Japan Production Monthly Growth'] = (Prod_t - Prod_t1)/Prod_t1
df_both.drop('Industrial Production t+1', inplace=True, axis=1)



Create a variable for yearly production growth. Do this by taking the production for time period 12 using .shift(12) and production for time period 0. Annual roduction growth is the growth in production for every 12 periods so divide the difference in production for the period that is twelve months later by the production in period 0. Finally, use .drop() to remove the variable created for production in period 12. This method will repeat the process for all  time periods across the 30 years and give the growth in each month relative to 12 months earlier.

In [ ]:
#Create a variable for yearly production growth. Description in line above
df_both['Industrial Production t+12'] = df_both['Japan Industrial Production'].shift(12)
Prod_t = df_both['Japan Industrial Production']
Prod_t12 = df_both['Industrial Production t+12']
df_both['Japan Production Yearly Growth'] = (Prod_t - Prod_t12)/Prod_t12
df_both.drop('Industrial Production t+12', inplace=True, axis=1)




In [ ]:
#Create a variable for an index of the value of Japan's international reserves (value of reserves at January 1960 = 100)
Japan_International_reserve_1960M1 = df_Japan.iloc[0,3]
df_both['Japan International Reserve index'] = df_both['Japan International Reserves and Liquidity']/Japan_International_reserve_1960M1*100
df_both.drop('Japan International Reserves and Liquidity', inplace=True, axis=1)

In [ ]:
#Create a variable for Log USA CPI by taking np.log() of US CPI
df_both['Log USA CPI'] = np.log(df_both['USA CPI'])


Create a variable for USA Inflation Monthly Change. Do this by taking the CPI for time period 1 using .shift() and CPI for time period 0. Inflation is the growth in CPI between two consecutive periods so divide the difference in CPI for the two periods by the CPI in period 0. Finally, use .drop() to remove the variable created for CPI in period 1. This method will repeat the process for all 2 consecutive time periods across the 30 years

In [ ]:
#Create a variable for USA Inflation monthly. Description in line above
df_both['USA CPI t+1'] = df_both['USA CPI'].shift()
USCPI_t = df_both['USA CPI']
USCPI_t1 = df_both['USA CPI t+1']
df_both['USA Inflation Monthly Change'] = (USCPI_t - USCPI_t1)/USCPI_t1
df_both.drop('USA CPI t+1', inplace=True, axis=1)




In [ ]:
#Create a variable for an index of the value of USA international reserves (value of reserves at January 1960 = 100)
USA_International_reserve_1960M1 = df_USA.iloc[0,1]
df_both['USA International Reserve index'] = df_both['USA International Reserves and Liquidity']/USA_International_reserve_1960M1*100
df_both.drop('USA International Reserves and Liquidity', inplace=True, axis=1)

In [ ]:
#Create a variable for Inflation difference between US and Japan by subtracting USA inflation change from Japan inflation change. We will use this variable later on
df_both['Inflation Difference'] = df_both['Japan Inflation Monthly Change'] - df_both['USA Inflation Monthly Change']

Firstly, calculate real exchange rate of Japan and assign this to variable Real ER by doing (Japan CPI x Nominal exchange rate per US dollar)/USA CPI.

We can then find monthly growth in real exchange rates by using np.log() and .diff() to take the log of real ER and then difference it with periods=1 to find change between each consecutive month as the growth equals change in logs.

In [ ]:
#Create a variable for Japan monthly real exchange exchange rate growth. Description above

df_both['Real ER'] = (df_both["Japan CPI"]*df_both["Exchange Rates Per U.S. Dollar"])/df_both["USA CPI"]
df_both['Japan Monthly Real ER Growth'] = (np.log(df_both['Real ER'])).diff()




To find outliers we begin by: 1) Creating a list of all the variable headers in the dataset and assign to column_titles variable.

2) Using a for loop, for each variable header in column title, we work out the upper and lower limit for which we identify outliers. We calculate the interquartile range using the 75th percentile subtract 25th percentile for each column. To find the upper limit, we do 3rd quartile + IQR. For lower limit, we do 1st quartile - IQR since this is the mathematical definitions for outliers

3) We then use a for loop within the original loop to loop through every value in each column. If the datapoint value is above the upper limit or below the lower limit then it is an outlier so we add it to a set

4) If the set has small number of implausibly extreme observations then we can add them to the variable of outliers_library which is a collection of all outliers and we use np.where() to identify the values which are above the upper limit or below the lower limit

5) We use .append() to add all outliers to this library so that we can view all of the outliers

6) Create another for loop to loop through each outlier in outliers_library and we go through the entire library using range(len)) as it needs to be an integer and not a tuple. We locate each outlier and call it x and use .iloc[x,1:] to locate every outlier and then assign it to a missing value as np.nan

The output lists the index of the outliers for each variable

To interpolate:

1) Use the df.interpolate() function and then use df.fillna() to replace the missing values with the average of surrounding values going from bottom to top using backfill method

Second method and analysis of outliers below this method.

In [ ]:
#Find and remove outliers. Interpolate missing values. Description in line above
variable_titles = (list(df_both))
variable_titles = variable_titles[1:]
set = 0
store = []

df_zscore=(df_both - df_both.mean(numeric_only=True))/df_both.std(numeric_only=True)
outliers = (df_zscore >3)|(df_zscore < -3)
df_both[outliers]=np.nan

df_both.interpolate(method = 'linear', inplace = True)
df_both.fillna(method = 'backfill', inplace = True)

    
    

We can also use boxplots to identify outliers. A boxplot was created for each variable with an outlier being one that is located outside of the whiskers. This shows that the following variables have outliers: Nominal ER Growth, Real ER Growth, Japan Inflation Monthly Change, USA Inflation Monthly Change, Production Monthly Growth, Production Yearly Growth which confirms what the above method showed.

From the code above that identifies outliers, the variables with less than 5 outliers are: Japan production monthly growth and Japan production yearly growth and Japan inflation monthly change and inflation difference

In [ ]:
#Boxplot method for outliers to identify which variables they are located in. Description above
plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Industrial Production'], color='blue')
plt.ylabel('Value')
plt.xlabel('Japan Industrial Production')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Exchange Rates Per U.S. Dollar'], color='purple')
plt.ylabel('Value')
plt.xlabel('Exchange Rates Per U.S. Dollar')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan CPI'], color='cyan')
plt.ylabel('Value')
plt.xlabel('Japan CPI')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['USA CPI'], color='magenta')
plt.ylabel('Value')
plt.xlabel('USA CPI')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['USA Inflation Monthly Change'], color='green')
plt.ylabel('Value')
plt.xlabel('USA Inflation Monthly Change')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Log Exchange Rate'], color='orange')
plt.ylabel('Value')
plt.xlabel('Log Exchange Rate')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Monthly Nominal ER Growth'], color='red')
plt.ylabel('Value')
plt.xlabel('Japan Monthly Nominal ER Growth')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Monthly Real ER Growth'], color='red')
plt.ylabel('Value')
plt.xlabel('Japan Monthly Real ER Growth')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Inflation Monthly Change'], color='white')
plt.ylabel('Value')
plt.xlabel('Japan Inflation Monthly Change')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Log Industrial Production'], color='yellow')
plt.ylabel('Value')
plt.xlabel('Log Industrial Production')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Production Monthly Growth'], color='black')
plt.ylabel('Value')
plt.xlabel('Japan Production Monthly Growth')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan International Reserve index'], color='purple')
plt.ylabel('Value')
plt.xlabel('Japan International Reserve index')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Japan Production Yearly Growth'], color='green')
plt.ylabel('Value')
plt.xlabel('Japan Production Yearly Growth')
plt.show()

plt.figure(figsize=(4,4))
sns.boxplot(data=df_both['Inflation Difference'], color='green')
plt.ylabel('Value')
plt.xlabel('Inflation Difference')
plt.show()

Now plot time series graphs for all of the variables. Use figure.() to choose a size that makes the graphs clear and easy to read. Use .title(), .xlabel(), .legend() to add titles and labels to the graph to make all parts of the graph understandable and clear. Use .axvline() to highlight the time at which Japan left the Bretton Woods system with a dashed line and a key in the legend. We also added a line for when the Nixon Shock occured because this helps us understand why there was fluctuations in macroeconomic aggregates before the exit but not throughout the bretton woods period and differentiates the impact of the Nixon shock from any outliers.

In [ ]:
#Time series graph of the monthly growth in nominal exchange rates of Japan, versus the US dollar
plt.figure(figsize = (15,10))
plt.plot(df_both['Date'], df_both["Japan Monthly Nominal ER Growth"], label = "Japan Nominal ER Growth")
plt.title("Japan Monthly Nominal Exchange Rate Growth in 1960-90")
plt.axvline(pd.to_datetime("1973-2-1"), color = "red", linestyle = "dashed", label = "Japan Bretton Woods Exit")
plt.axvline(pd.to_datetime("1971-8-1"), color = "green", linestyle = "dashed", label = "Nixon Shock")
plt.xlabel("Year")
plt.ylabel("Monthly Nominal Exchange Rate Growth (in %)")
plt.legend()
plt.show()

In [ ]:
##Time series graph of the monthly growth in real exchange rates of Japan, versus the US dollar
plt.figure(figsize = (15,10))
plt.plot(df_both['Date'], df_both["Japan Monthly Real ER Growth"], label = "Japan Real ER Growth")
plt.title("Japan Monthly Real Exchange Rate Growth in 1960-90")
plt.axvline(pd.to_datetime("1973-2-1"), color = "red", linestyle = "dashed", label = "Japan Bretton Woods Exit")
plt.axvline(pd.to_datetime("1971-8-1"), color = "green", linestyle = "dashed", label = "Nixon Shock")
plt.xlabel("Year")
plt.ylabel("Monthly Real Exchange Rate Growth (in %)")
plt.legend()
plt.show()

In [ ]:
#Time series graph of the monthly indexed value of international reserves, Japan and USA
plt.figure(figsize = (15,10))
plt.plot(df_both['Date'], df_both['Japan International Reserve index'], label = 'Japan International Reserve Index')
plt.plot(df_both['Date'], df_both['USA International Reserve index'], label = 'USA International Resverve Index')
plt.title('International Reserve Index of Japan and USA, 1960-90 (value of reserves at January 1960 = 100)')
plt.axvline(pd.to_datetime("1973-2-1"), color = "red", linestyle = "dashed", label = "Japan Bretton Woods Exit")
plt.axvline(pd.to_datetime("1971-8-1"), color = "green", linestyle = "dashed", label = "Nixon Shock")
plt.xlabel("Year")
plt.ylabel("International Reserve Index Value")
plt.legend()
plt.show()

In [ ]:
#Time series graph of monthly inflation
plt.figure(figsize = (15,10))
plt.plot(df_both['Date'], df_both["Japan Inflation Monthly Change"], label = "Japan Inflation")
plt.title("Japan Inflation in 1960-90")
plt.axvline(pd.to_datetime("1973-2-1"), color = "red", linestyle = "dashed", label = "Japan Bretton Woods Exit")
plt.axvline(pd.to_datetime("1971-8-1"), color = "green", linestyle = "dashed", label = "Nixon Shock")
plt.xlabel("Year")
plt.ylabel("Inflation (in %)")
plt.legend()
plt.show()

In [ ]:
#Time series graph of the growth in industrial production versus 12 months ago
plt.figure(figsize = (15,10))
plt.plot(df_both['Date'], df_both["Japan Production Yearly Growth"], label = "Japan Annual Production Growth")
plt.title("Japan Production Yearly Growth in 1960-90")
plt.axvline(pd.to_datetime("1973-2-1"), color = "red", linestyle = "dashed", label = "Japan Bretton Woods Exit")
plt.axvline(pd.to_datetime("1971-8-1"), color = "green", linestyle = "dashed", label = "Nixon Shock")
plt.xlabel("Year")
plt.ylabel("Production Growth (in %)")
plt.legend()
plt.show()

In [ ]:
#Time series graph of the monthly growth in industrial production
plt.figure(figsize = (15,10))
plt.plot(df_both['Date'], df_both["Japan Production Monthly Growth"], label = "Japan Monthly Production Growth")
plt.title("Japan Production Monthly Growth in 1960-90")
plt.axvline(pd.to_datetime("1973-2-1"), color = "red", linestyle = "dashed", label = "Japan Bretton Woods Exit")
plt.axvline(pd.to_datetime("1971-8-1"), color = "green", linestyle = "dashed", label = "Nixon Shock")
plt.xlabel("Year")
plt.ylabel("Production Growth (in %)")
plt.legend()
plt.show()



Firstly, identify the periods before and after the country left the Bretton Woods, leaving one year either side. This is done by using .loc[] indexing and selecting the rows required for these dates. Assign this to variables before_exit and after_exit. Index 131 is 10 months before the exit so we use .loc[0:131] and index 151 is 10 months after.

Then create the variable Inflation Difference by calculating the difference between Japan and US inflation which we have already done in code above

Assign all variables and column titles in the dataframe to 'variables'

Create column_1 and column_2 which calculates the standard deviation of all of the values in each of the 'variables' columns by using .std() and this calculates the standard deviations for before_exit and after_exit respectively

Create a table using pd.DataFrame which consists of column_1 and column_2 so shows the standard deviations for each variables before and after. Using .transpose() to present it clearly

Calculate the ratio of before and after exit

In [ ]:
#Calculate standard deviations using the method in the line above
before_exit = df_both.loc[0:131]
after_exit = df_both.loc[151:]
variables = ['Japan Monthly Nominal ER Growth','Japan Monthly Real ER Growth','Japan Inflation Monthly Change','Inflation Difference','Japan Production Yearly Growth']
column_1 = before_exit[variables].std()
column_2 = after_exit[variables].std()
Table = pd.DataFrame([column_1,column_2], index = ['Before Bretton Woods Exit', 'After Bretton Woods Exit']).transpose()
Table['Ratio'] = Table['After Bretton Woods Exit']/Table['Before Bretton Woods Exit']
Table
                        

In [ ]:
#Increase max number of rows displayed to observe entire dataframe
pd.set_option('display.max_rows', 500)

In [ ]:
#Rearrange columns to make the headers for Japan and USA separate
df_both = df_both[['Date', 'Japan Industrial Production', 'Exchange Rates Per U.S. Dollar', 'Japan CPI', 'Japan Log Exchange Rate', 'Japan Log Industrial Production', 'Japan Production Monthly Growth', 'Japan Production Yearly Growth', 'Japan Monthly Nominal ER Growth', 'Japan Monthly Real ER Growth', 'Japan Inflation Monthly Change', 'Japan International Reserve index', 'USA CPI', 'Log USA CPI', 'USA Inflation Monthly Change', 'USA International Reserve index', 'Inflation Difference']]

In [ ]:
df_both